### Prepare Dataset

In [1]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
path = "/data/nvme6n1p1/adal_workspace/small_llm/datasets/sft_dataset_conversation"
df_train = pd.read_csv(f"{path}/everyday-conversations_train-kk.csv")
df_test = pd.read_csv(f"{path}/everyday-conversations_test-kk.csv")

In [2]:
from datasets import Dataset, DatasetDict, concatenate_datasets
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

ds_dict = DatasetDict({
    "train":ds_train.select_columns(["messages"]),
    "test":ds_test.select_columns(["messages"])
})

/root/anaconda3/envs/atune/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds_dict

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 119
    })
})

In [4]:
def apply_chatting_template_full(sample):
    sample["text"]=sample['messages'].replace("<#>", "<|im_start|>user:").replace("<*>", "<|im_end|>\n<|im_start|>assistant:") + "<|im_end|>"
    return sample

In [5]:
ds_input = ds_dict.map(apply_chatting_template_full).select_columns(["text"])

Map: 100%|██████████| 119/119 [00:00<00:00, 24442.81 examples/s]


In [6]:
print(ds_input['train'][0]['text'])

 <|im_start|>user:
Сәлеметсіз бе
<|im_end|>
<|im_start|>assistant:
Сәлеметсіз бе! Бүгін мен сізге қалай көмектесе аламын?
<|im_start|>user:
Мен келесі демалысыма жағажай курортын іздеп жүрмін. Кейбір танымал жағажайларды ұсына аласыз ба?
<|im_end|>
<|im_start|>assistant:
Кейбір танымал жағажай курорттарына Гавайидегі Мауи, Мальдив аралдары және Багам аралдары кіреді. Олар әдемі жағажайларымен және мөлдір суларымен танымал.
<|im_start|>user:
Бұл керемет естіледі. Кариб теңізінде отбасылар үшін қолайлы курорттар бар ма?
<|im_end|>
<|im_start|>assistant:
Иә, Кариб теңізіндегі отбасыларға қолайлы курорттар үшін Туркс және Кайкос аралдары және Барбадос тамаша таңдау болып табылады. Олар барлық жастағы адамдарға арналған бірқатар іс-шаралар мен ыңғайлылықтарды ұсынады.
<|im_start|>user:
Жарайды, мен оларды қарастырамын. Ұсыныстарыңыз үшін рақмет!
<|im_end|>
<|im_start|>assistant:
Қош келдіңіз. Демалыс үшін тамаша курорт табасыз деп үміттенемін.
<|im_end|>


### Model preparation

In [7]:
path = "/data/nvme6n1p1/adal_workspace/small_llm/models/small_lm_test1"

from transformers import LlamaForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LlamaForCausalLM.from_pretrained(path).to(device)
tokenizer = AutoTokenizer.from_pretrained(path)

In [8]:
tokenized_output = tokenizer(
    ds_input['train'][0]['text'], 
    padding="max_length",  # You can also use padding=True for dynamic padding
    truncation=True,  # Ensures the text does not exceed max length
    max_length=512,  # Adjust based on your model
    return_tensors="pt"  # Returns PyTorch tensors (or use "tf" for TensorFlow)
)

print(tokenized_output)

{'input_ids': tensor([[  237,     1,   518,   316,    42,   215, 22582,  1754, 14572,  8150,
           215,     2,   215,     1,  1262, 24939,    42,   215, 22582,  1754,
         14572,  8150,    17, 19726,   691, 28952,  5327,  6058,   719, 34041,
            47,   215,     1,   518,   316,    42,   215,  8842,  5079,  8659,
          4946,   471, 36521, 39895,  1399, 20160, 29804,   379,    30, 13514,
          5550, 36521,  2852, 31680, 26133,   701,    47,   215,     2,   215,
             1,  1262, 24939,    42,   215, 31024,  5550, 36521, 39895, 18142,
         22432,   431, 35184,   531, 37462,    28, 26594, 12506, 27628,   456,
          3747, 44306, 27628,  8840,    30,  4648, 20158, 36521, 17430,   456,
         44413,  6770, 32967,  5550,    30,   215,     1,   518,   316,    42,
           215,  4612, 14464,  7684,  2053,    30, 42537,  5397,   561, 40009,
           820, 14896, 39895, 11939,   964,   835,    47,   215,     2,   215,
             1,  1262, 24939,    42,  

In [12]:
# Configure the SFTTrainer
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

sft_config = SFTConfig(
    report_to=None, # To disable wandb
    output_dir="/data/nvme6n1p1/adal_workspace/small_llm/models/chatting_llm/try_1",
    adam_beta1=0.9,
    adam_beta2=0.95,
    max_seq_length=1024,
    # max_steps=141,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=8,  # Set according to your GPU memory capacity
    learning_rate=3e-6,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=100,  # Frequency of evaluation
    num_train_epochs=2,
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

/root/anaconda3/envs/atune/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds_input["train"],
    tokenizer=tokenizer,
    eval_dataset=ds_input["test"],
)

/tmp/ipykernel_682649/198691415.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Truncating eval dataset: 100%|██████████| 119/119 [00:00<00:00, 4346.43 examples/s]


In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: mamubieke-parehati (mamubieke-parehati-ISSAI) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
100,1.355600,1.362775
200,1.256700,1.251368
300,1.161300,1.210384
400,1.173300,1.194203
500,1.149100,1.186450


TrainOutput(global_step=566, training_loss=1.2867608087222904, metrics={'train_runtime': 386.8576, 'train_samples_per_second': 11.684, 'train_steps_per_second': 1.463, 'total_flos': 9979924473114624.0, 'train_loss': 1.2867608087222904})

### Run model's inference

In [15]:
prompt = "\nСәлем\n"

messages = [{'role':'user', 'content':prompt}]

In [16]:
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [17]:
formatted_prompt_2 = formatted_prompt.replace("user : ", "user:\n") + "<|im_start|>assistant:\n"

inputs_2 = tokenizer(formatted_prompt_2, return_tensors="pt").to(device)

In [18]:
print(formatted_prompt_2)

<|im_start|>user

Сәлем
<|im_end|>
<|im_start|>assistant:



In [20]:
tokenizer.encode("<|im_end|>\n<|im_start|>")

[2, 215, 1]

In [19]:
from transformers import StoppingCriteria
# https://github.com/huggingface/trl/issues/921
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = [2, 215, 1]):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids

In [24]:
new_model = LlamaForCausalLM.from_pretrained("/data/nvme6n1p1/adal_workspace/small_llm/models/chatting_llm/try_1/checkpoint-566").to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.34it/s]


In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
streamer = TextStreamer(tokenizer)
ouputs  = new_model.generate(**inputs_2,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.3,
    stopping_criteria=[EosListStoppingCriteria()],
    # streamer=streamer, 
    repetition_penalty=1.17, 
    # top_p=1.0,
    eos_token_id=tokenizer.encode("<|im_start|>"))

In [27]:
ouputs_2  = new_model.generate(**inputs_2, max_new_tokens=100, do_sample=True, eos_token_id=tokenizer.encode("<|im_end|>"), repetition_penalty=2.07, temperature=0.1, top_p=1.0, streamer=streamer)

<|im_start|>user

Сәлем
<|im_end|>
<|im_start|>assistant:
Қош келдіңіз! );- деп жауап берді. | (қос нүктеден кейін) және теруді жалғастырыңыз). Бұл көмектеседі, рахмет; қажет емес.)>><h2 ><p class="title" id="" name = "Мысықтың аты"></a href_lengths` </heading style “Tail Length”) # Мысалдар мен кеңестер үшін атауды пайдаланыңыз немесе оны өзгертіңіз): мысықтар туралы


In [62]:
import json
from argparse import ArgumentParser
from pathlib import Path
from typing import Literal, Optional
from transformers import AutoTokenizer, LlamaForCausalLM
from transformers import LlamaConfig as HFLlamaConfig

TEST_PROMPT = "Қазақстан туралы не білесін?"

def check_converted_model_generation(save_path: Path):
    """Loads a huggingface model and tokenizer from `save_path` and
    performs a dummy text generation."""

    tokenizer = AutoTokenizer.from_pretrained(save_path)
    messages = [{'role':'user', 'content':TEST_PROMPT}]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    formatted_prompt_2 = formatted_prompt.replace("user : ", "user:\n") + "<|im_start|>assistant:\n"
    input_ids = tokenizer(formatted_prompt_2, return_tensors="pt")["input_ids"].cuda()
    print("Inputs:", tokenizer.batch_decode(input_ids))

    model = LlamaForCausalLM.from_pretrained(save_path).cuda().bfloat16()
    out = model.generate(input_ids, max_new_tokens=1000, do_sample=True, temperature=0.3, top_p=0.98, repetition_penalty=1.17)
    
    print("Generation (converted): ", tokenizer.decode(out[0], skip_special_tokens=True))
    # print("Generation (Not-converted): ", out[0])

In [69]:
TEST_PROMPT = "«Қазақстан отар болып келді және солай болып қалды» деп айтқан қайраткер?"
check_converted_model_generation("/data/nvme6n1p1/adal_workspace/small_llm/models/chatting_llm/try_1/checkpoint-566")

Inputs: ['<|im_start|>user\n«Қазақстан отар болып келді және солай болып қалды» деп айтқан қайраткер?<|im_end|>\n<|im_start|>assistant:\n']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.99it/s]


Generation (converted):  user
«Қазақстан отар болып келді және солай болып қалды» деп айтқан қайраткер?
assistant:
Ол 1920 жылдары Кеңес өкіметіне қарсы шыққан қазақ жастарын қолдаған, бірақ кейін ол «халық жауы» деген айыппен тұтқындалып, ату жазасына кесілген.


In [97]:
TEST_PROMPT = "Сингапур туралы айтып берші?"

tokenizer = AutoTokenizer.from_pretrained("/data/nvme6n1p1/adal_workspace/small_llm/models/chatting_llm/try_1/checkpoint-566")
messages = [{'role':'user', 'content':TEST_PROMPT}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
formatted_prompt_2 = formatted_prompt.replace("user : ", "user:\n") + "<|im_start|>assistant:\n"

In [105]:
from openai import OpenAI
client = OpenAI(
    base_url="http://0.0.0.0:8009/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="/data/nvme6n1p1/adal_workspace/small_llm/models/chatting_llm/try_1/checkpoint-566",
  messages=[
        {"role": "user", "content": formatted_prompt}
    ],
  stream=True,
  temperature=0.4,
  top_p=0.95,
  stop=["<|endoftext|>","<|im_end|>", "<|im_start|>"],
    extra_body={
        "skip_special_tokens": False,
        # "repetition_penalty":1.17
    },
)
cool = []
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        cool.append(chunk.choices[0].delta.content)

print("".join(cool))

Сингапур - Оңтүстік-Шығыс Азиядағы аралдық мемлекет. Ол 1963 жылы 9 желтоқсанда Ұлыбританиядан тәуелсіздік алды. Ол өзінің ерекше мәдениеті мен тілімен, таңғажайып жағажайларымен және жанды түнгі өмірімен танымал.

